In [1]:
import speech_recognition as sr
import pyttsx3
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import webbrowser
import datetime
import os
import time
import threading
import tkinter as tk
from tkinter import messagebox

# ===============================
# SETTINGS
# ===============================
MIC_ID = 1
SAMPLE_RATE = 44100
RECORD_SECONDS = 5
TEMP_WAV = "input.wav"

assistant_running = False  # Global flag

# ===============================
# TEXT TO SPEECH
# ===============================
engine = pyttsx3.init()
engine.setProperty("rate", 175)
engine.setProperty("volume", 1.0)

def speak(text):
    print("Assistant:", text)
    engine.say(text)
    engine.runAndWait()

# ===============================
# LISTEN FUNCTION
# ===============================
def listen():
    recording = sd.rec(
        int(RECORD_SECONDS * SAMPLE_RATE),
        samplerate=SAMPLE_RATE,
        channels=1,
        device=MIC_ID,
        dtype="float32"
    )
    sd.wait()

    # Normalize
    recording = recording / np.max(np.abs(recording))

    # Convert to PCM int16
    pcm_audio = np.int16(recording * 32767)
    wav.write(TEMP_WAV, SAMPLE_RATE, pcm_audio)

    r = sr.Recognizer()
    with sr.AudioFile(TEMP_WAV) as source:
        r.adjust_for_ambient_noise(source, duration=0.6)
        audio = r.record(source)

    try:
        command = r.recognize_google(audio)
        print("You said:", command)
        return command.lower()
    except:
        return ""

# ===============================
# COMMAND PROCESSOR
# ===============================
def process_command(command):
    global assistant_running

    if not command:
        return

    if "exit" in command or "quit" in command or "stop" in command:
        speak("Goodbye. Assistant stopped.")
        assistant_running = False
        return

    elif "hello" in command:
        speak("Hello! How can I help you?")

    elif "time" in command:
        t = datetime.datetime.now().strftime("%I:%M %p")
        speak(f"The time is {t}")

    elif "open google" in command:
        speak("Opening Google")
        webbrowser.open("https://www.google.com")

    elif "open youtube" in command:
        speak("Opening YouTube")
        webbrowser.open("https://www.youtube.com")

    elif "open notepad" in command:
        speak("Opening Notepad")
        os.system("notepad")

    else:
        speak("Sorry, I did not understand that")

# ===============================
# ASSISTANT LOOP (THREAD)
# ===============================
def assistant_loop():
    global assistant_running
    speak("Voice assistant started")

    while assistant_running:
        command = listen()
        process_command(command)
        time.sleep(0.4)

# ===============================
# GUI CONTROLS
# ===============================
def start_assistant():
    global assistant_running

    if assistant_running:
        messagebox.showinfo("Info", "Assistant is already running")
        return

    assistant_running = True
    threading.Thread(target=assistant_loop, daemon=True).start()

def stop_assistant():
    global assistant_running
    assistant_running = False
    speak("Assistant stopped")

# ===============================
# GUI WINDOW
# ===============================
root = tk.Tk()
root.title("Voice Assistant")
root.geometry("350x250")
root.resizable(False, False)

title = tk.Label(root, text="Custom Voice Assistant", font=("Arial", 16, "bold"))
title.pack(pady=15)

start_btn = tk.Button(
    root, text="Start Assistant", font=("Arial", 12),
    width=20, bg="green", fg="white", command=start_assistant
)
start_btn.pack(pady=10)

stop_btn = tk.Button(
    root, text="Stop Assistant", font=("Arial", 12),
    width=20, bg="red", fg="white", command=stop_assistant
)
stop_btn.pack(pady=10)

exit_btn = tk.Button(
    root, text="Exit Program", font=("Arial", 11),
    width=20, command=root.destroy
)
exit_btn.pack(pady=10)

root.mainloop()


Assistant: Voice assistant started
You said: open
Assistant: Sorry, I did not understand that
You said: open Notepad
Assistant: Opening Notepad
You said: Google
Assistant: Sorry, I did not understand that
You said: open
Assistant: Sorry, I did not understand that
You said: open Google open Google
Assistant: Opening Google
You said: what is the time
Assistant: The time is 11:57 PM
You said: what is the time
Assistant: The time is 11:58 PM
You said: time
Assistant: The time is 11:58 PM
You said: hello
Assistant: Hello! How can I help you?
Assistant: Assistant stopped
Assistant: Assistant stopped
Assistant: Assistant stopped
